In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
data_dir = Path('C:/Users/user/Downloads/Project_THDL/Beer data')

In [ ]:
df_reviews = pd.read_csv(data_dir / 'beer_reviews/beer_reviews.csv')
df_reviews.info

In [ ]:
df_reviews.head(10)

In [ ]:
df_scores = df_reviews.drop(['brewery_id', 
                 'review_time', 
                 'review_profilename', 
                 'beer_style', 
                 'beer_beerid', 
                 'beer_abv'],axis=1).groupby(by=['brewery_name', 
                                                 'beer_name'], as_index=False).mean()

df_scores_count = df_reviews.drop(['brewery_id', 
                 'review_time', 
                 'review_profilename', 
                 'beer_style', 
                 'beer_beerid', 
                 'beer_abv'],axis=1).groupby(by=['brewery_name', 
                                                 'beer_name'], as_index=False).count()

df_scores.head(10)


In [ ]:
df_scores_count = df_scores_count['review_overall']

df_scores['number_of_reviews'] = df_scores_count
df_scores.head(10)

In [ ]:
df_profile = pd.read_csv(data_dir / 'beer_data_set/beer_data_set.csv')
df_profile.info()

In [ ]:
df_profile[df_profile['Name'].isnull()]

In [ ]:
df_profile.loc[[1803, 2150],'Name'] = df_profile.loc[[1803, 2150],'Brewery']

In [ ]:
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)

slice_ = 'brewery_review_exists'

def highlight_indicator(val):
    pink = 'background-color: pink' if val < 1 else ''
    return pink

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

In [ ]:
brewery_no_scores = pd.DataFrame(df_profile[df_profile['brewery_review_exists']==0]['Brewery'].unique()).set_axis(['Brewery'], axis=1)
brewery_no_scores

In [ ]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(query=x, choices=s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [ ]:
# Create new column on `df_scores` indicating whether the brewery name exists on `df_profile`
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

# Create new data frame (`brewery_no_profile`) listing breweries on `df_scores` with no exact match on `df_profile`
brewery_no_profile = pd.DataFrame(df_scores[df_scores['brewery_profile_exists']==0]['brewery_name'].unique()).set_axis(['Brewery'], axis=1)

In [ ]:
# # (Uncomment to let pandas display all rows and column content for all data frames)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

# # (Default setting)
# pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_colwidth', 50)

In [ ]:
brewery_no_scores.head(5)


In [ ]:
brewery_no_profile.head(5)

In [ ]:
# # Get fuzzy matches for 543 breweries
# # Warning: Takes a long time to run
fuzzy_match_brewery = fuzzy_merge(brewery_no_scores, brewery_no_profile, 'Brewery', 'Brewery', threshold=88, limit=1)
fuzzy_match_brewery.to_csv(data_dir/ 'Brewery Fuzzy Match List.csv', index=False)

In [ ]:
#Load table containing saved brewery name matches ("Brewery Name Fuzzy Match List.csv")
fuzzy_match_brewery = pd.read_csv(data_dir / "Brewery Fuzzy Match List.csv")
fuzzy_match_brewery

In [ ]:
# Save table as dictionary
fuzzy_match_dict = dict(zip(fuzzy_match_brewery['Brewery'], fuzzy_match_brewery['matches']))

# Replace relevant brewery names in `df_profile`
df_profile['Brewery'] = df_profile['Brewery'].replace(fuzzy_match_dict)

In [ ]:
# Correcting 'brewery_review_exists' indicator column values on `df_profile` after using fuzzy matches
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)

# Correcting 'brewery_profile_exists' indicator column values on `df_scores` after using fuzzy matches
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

In [ ]:
# Remove all brews from breweries that are not listed in either `df_profile` or `df_scores`
df_scores.drop(df_scores[df_scores['brewery_profile_exists']==0].index, axis=0, inplace=True)
df_profile.drop(df_profile[df_profile['brewery_review_exists']==0].index, axis=0, inplace=True)

In [ ]:
# Create new identifier column in `df_scores` called 'beer_name_full'
# Fill column by concatenating `brewery_name` and `beer_name`
df_scores['beer_name_full'] = df_scores['brewery_name'] + ' ' + df_scores['beer_name']

# Check if all generated brew names in df_scores are unique:
df_scores['beer_name_full'].nunique() == len(df_scores.index)

In [ ]:
# Create new identifier column in `df_profile` called 'Beer Name (Full)' 
# Fill column by concatenating `Brewery` and `Name`
df_profile['Beer Name (Full)'] = df_profile['Brewery'] + ' ' + df_profile['Name']

# Check if all generated brew names in `df_profile` are unique:
df_profile['Beer Name (Full)'].nunique() == len(df_profile.index)

In [ ]:
# Check duplicated brew name
df_profile[df_profile['Beer Name (Full)'].duplicated()]

In [ ]:
# Remove duplicate row containing error (missing data)
df_profile.drop(4056, inplace=True)
# Remove duplicate row containing error (missing data)
df_profile.drop(4033, inplace=True)

In [ ]:
# Create new column on `df_profile` indicating whether the complete brew name exists on `df_scores`
df_profile['beer_review_exists'] = np.where(df_profile['Beer Name (Full)'].isin(list(df_scores['beer_name_full'])), 1, 0)


# Formatting for better display
slice_ = 'beer_review_exists'

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

In [ ]:
# Create new data frame (`beer_no_scores`) listing brews on `df_profile` with no exact match on `df_scores`
beer_no_scores = pd.DataFrame(df_profile[df_profile['beer_review_exists']==0]['Beer Name (Full)']).set_axis(['Beer Name (Full)'], axis=1)
beer_no_scores

In [ ]:
# Create new column on `df_scores` indicating whether the complete brew name exists on `df_profile`
df_scores['beer_profile_exists'] = np.where(df_scores['beer_name_full'].isin(list(df_profile['Beer Name (Full)'])), 1, 0)

# Create new data frame (`beer_no_profile`) listing brews on `df_scores` with no exact match on `df_profile`
beer_no_profile = pd.DataFrame(df_scores[df_scores['beer_profile_exists']==0]['beer_name_full']).set_axis(['beer_name_full'], axis=1)

In [ ]:
# # Get fuzzy matches for 2460 brews
# # Warning: Takes a long time to run
#fuzzy_match_beer_name = fuzzy_merge(beer_no_scores, beer_no_profile, 'Beer Name (Full)', 'beer_name_full', threshold=87, limit=1)
#fuzzy_match_beer_name.to_csv(data_dir/ 'Beer Name Fuzzy Match List (my).csv', index=False)

In [ ]:
fuzzy_match_beer_name = pd.read_csv(data_dir / 'Beer Name Fuzzy Match List (my).csv')
fuzzy_match_beer_name